In our human annnotations we found severe disagreement between annotators. In this file
- I perform some preliminary explorations on the nature of this disagreement
- I run GPT-4 labelling on all other samples

#### Explorations

In [1]:
import pandas as pd
import json

The files in this repository
1. new_annotated_sample.pkl: the ~1200 samples with labels generated by us
2. annotated_test.pkl: exactly the same
3. annotated_val.pkl: labels by law students
4. disagreement_sample.json: the 50 overlapping samples, where almost 50% disagreement is

In [2]:
f = open('disagreement_sample.json')
data = json.load(f)

In [3]:
labels = [sample["label"] for sample in data]
sums = [sum(sample) for sample in labels]

for i in set(sums):
    print(f"{i} occurs {sums.count(i)} times")

1 occurs 9 times
2 occurs 6 times
3 occurs 4 times
4 occurs 5 times


In [8]:
# finding out how often each labeller assigned label 1
# all around 50%, except labeller 2
labels = pd.DataFrame(labels)
(labels.sum(axis=0)/len(labels)).round(2)

0    0.50
1    0.08
2    0.54
3    0.54
4    0.54
dtype: float64

#### Obtain explanations for entire dataset samples

In [4]:
df = pd.read_pickle("new_annotated_sample.pkl")
df2 = pd.read_pickle("annotated_test.pkl")
df3 = pd.read_pickle("annotated_val.pkl")

In [2]:
# imports
import openai
import sys
sys.path.append('../99-other-project')
from gptClassifier2 import gptclassifier, standard_message

openai.api_key = "" # insert your api key

In [13]:
# execute classification
completions=[]
results = gptclassifier(df, standard_message, completions, timer_frequency=5)

Counter at 1
Counter at 6
Counter at 11
Counter at 16
Counter at 21
Counter at 26
Counter at 31
Counter at 36
Counter at 41
Counter at 46
Counter at 51
Counter at 56
Counter at 61
Counter at 66
Counter at 71
Counter at 76
Counter at 81
Counter at 86
Counter at 91
Counter at 96
Counter at 101
Counter at 106
Counter at 111
Counter at 116
Counter at 121
Counter at 126
Counter at 131
Counter at 136
Counter at 141
Counter at 146
Counter at 151
Counter at 156
Counter at 161
Counter at 166
Counter at 171
Counter at 176
Counter at 181
Counter at 186
Counter at 191
Counter at 196
Counter at 201
Counter at 206
Counter at 211
Waiting for 65s APIConnectionError
Counter at 216
Counter at 221
Counter at 226
Counter at 231
Counter at 236
Counter at 241
Counter at 246
Counter at 251
Counter at 256
Counter at 261
Counter at 266
Counter at 271
Counter at 276
Counter at 281
Counter at 286
Counter at 291
Counter at 296
Counter at 301
Counter at 306
Counter at 311
Counter at 316
Counter at 321
Counter at 3

KeyboardInterrupt: 

In [22]:
import pickle as pkl
with open('first_half.pkl', 'wb') as f:
    pkl.dump(completions, f)

In [20]:
f.close()

In [24]:
with open('first_half.pkl', 'rb') as f:
    new_df = pkl.load(f)

In [25]:
new_df

["Key indicators: '#YoungRock', 'NBC', '@SevenBucksProd'.\nReasoning: This post promotes a TV show. It again branding of the TV Studio @SevenBucksProd indicates commercial ties. Label: Potentially sponsored.",
 "Key indicators: '@priscavera'.\nReasoning: The post references the New York Fashion Week (@priscavera) and praises the designer as well. However, it does not seem like a paid promotion. Label: Likely not sponsored.",
 "Key indicators: 'Follow @wildbirdbaby', 'Tag friends', 'Share to stories'.\nReasoning: The post clearly promotes Wildbird, calling it 'baby brand' and is offering the chance to win some company items. These activities are often effective marketing techniques, therefore it is most likely a self advertisement. Label: Self advertisement.",
 'Key indicators: none.\nReasoning: There is no indication of self-promotion or promoting others, nor is there any mention of a business, company or product. Likely not sponsored. Label: Likely not sponsored.',
 'Key indicators: n